Data description:
> mock: mocked dataset with a small subset of images & labels \
> wangtest: original paper test set \
> wangtrain: original paper train set \
> train1 (split into a,b,c): ['H', 'X', 'L', 'Y'] \
> test 1: ['G'] \
> train2 (split into a,b,c): ['H', 'G', 'L', 'Y'] \
> test2: ['X'] \
> train3 (split into a,b,c): ['H', 'X', 'G'] \
> test3: ['L', 'Y']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_paths = {
    "train1": "/content/drive/Shareddrives/DL_Final_Project/shuffle/train1",
    "test1": "/content/drive/Shareddrives/DL_Final_Project/FINAL_DATASETS/test1",
    "train2": "/content/drive/Shareddrives/DL_Final_Project/shuffle/train2",
    "test2": "/content/drive/Shareddrives/DL_Final_Project/FINAL_DATASETS/test2",
    "train3": "/content/drive/Shareddrives/DL_Final_Project/shuffle/train3",
    "test3": "/content/drive/Shareddrives/DL_Final_Project/FINAL_DATASETS/test3",
    "train4": "/content/drive/Shareddrives/DL_Final_Project/FINAL_DATASETS/train4",
    "test4": "/content/drive/Shareddrives/DL_Final_Project/FINAL_DATASETS/test4"
}

In [ ]:
directory_dict = {
    "group1" :  "/content/drive/Shareddrives/DL_Final_Project/checkpoints1_new",
    "group2" :  "/content/drive/Shareddrives/DL_Final_Project/checkpoints2",
    "group3" :  "/content/drive/Shareddrives/DL_Final_Project/checkpoints3",
    "group4" :  "/content/drive/Shareddrives/DL_Final_Project/checkpoints4"
}

In [ ]:
# contains num_classes for each dataset -- test & train are the same
num_classes_dict = {
    "train1": 1777,
    "train2": 1656,
    "train3": 1628,
    "train4": 1588
}

In [ ]:
'''
Returns the args for the group
num_classes, train_data, test_data, directory
'''
def get_group_args(group):
  assert group in [1,2,3,4]
  if group==1:
    num_classes = num_classes_dict["train1"]
    train_data = dataset_paths["train1"]
    test_data = dataset_paths["test1"]
    directory = directory_dict["group1"]
  if group==2:
    num_classes = num_classes_dict["train2"]
    train_data = dataset_paths["train2"]
    test_data = dataset_paths["test2"]
    directory = directory_dict["group2"]
  if group == 3:
    num_classes = num_classes_dict["train3"]
    train_data = dataset_paths["train3"]
    test_data = dataset_paths["test3"]
    directory = directory_dict["group3"]
  if group == 4:
    num_classes = num_classes_dict["train4"]
    train_data = dataset_paths["train4"]
    test_data = dataset_paths["test4"]
    directory = directory_dict["group4"]
  return num_classes, train_data, test_data, directory


In [ ]:
num_classes, train_datapath, test_datapath, directory = get_group_args(1)
                                                # CHANGE THIS NUMBER ONLY WHEN SWITCHING GROUPS

In [ ]:
import tensorflow as tf
# ======================= LOAD DATASETS =======================
# load train set
train_dataset = tf.data.Dataset.load(train_datapath)
                                      # change train1 to other keys in dataset_paths

# load test set
test_dataset = tf.data.Dataset.load(test_datapath)
                                      # change test1 to other keys

# # uncomment to see shapes:
# for item in train_dataset.take(5):
#   print(item[0].shape)
#   print(item[1].shape)

# ^^ MICHELLE ADDED THE ABOVE LINES 5/2; DID NOT CHANGE BELOW
# ============================================================
# batching
batch_size = 256
train_dataset1 = train_dataset.batch(batch_size)
train_dataset1 = train_dataset1.prefetch(tf.data.experimental.AUTOTUNE)

test_dataset=test_dataset.batch(batch_size)
test_dataset1 = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

print(len(train_dataset1))
#print(num_classes)
print(len(test_dataset1))
#print(num_classes_test)


238
64


# Residual

The functionality of a residual block typically found in ResNet architectures. It includes convolutional layers, batch normalization, and possible adjustments in dimensions through 1x1 convolutions and stride alterations.

In [ ]:
from tqdm import tqdm
import os
import math
import json
import pandas as pd
import argparse
import random
import numpy as np
import sys
from datetime import datetime

In [ ]:
# Create a parser
parser = argparse.ArgumentParser(description='Train on HUST-OBS')

# Add expected arguments
parser.add_argument('--lr', '--learning-rate', default=0.001, type=float, metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--epochs', default=50, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--batch_size', default=256.0, type=float, metavar='N', help='mini-batch size')
parser.add_argument('--num_workers', default=24.0, type=float)
parser.add_argument('--wd', default=5e-4, type=float, metavar='W', help='weight decay')
parser.add_argument('--resume', default='', type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
# parser.add_argument('--results_dir', default='output', type=str, metavar='PATH', help='path to cache (default: none)')
parser.add_argument('--checkpoint_freq', type=int, default=5)
parser.add_argument('--seed', type=int, default=42)

# Check if the code is running in the IPython environment
if 'ipykernel' in sys.modules:
    # Assume a default setup; modify as needed or determine a way to set these via the notebook interface
    args = parser.parse_args(args=[])
else:
    args = parser.parse_args()

# Set random seed for reproducibility
tf.random.set_seed(args.seed)
np.random.seed(args.seed)
random.seed(args.seed)
os.environ['PYTHONHASHSEED'] = str(args.seed)

# Make results directory if it doesn't exist
# if args.results_dir == '':
#     args.results_dir = './cache-' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
if not os.path.exists(directory):
    os.makedirs(directory)

print(args)

Namespace(lr=0.001, epochs=50, batch_size=256.0, num_workers=24.0, wd=0.0005, resume='', checkpoint_freq=5, seed=42)


In [ ]:
class Residual(tf.keras.layers.Layer):
    def __init__(self, input_channels, min_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super(Residual, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(min_channels, kernel_size=1)
        self.conv2 = tf.keras.layers.Conv2D(min_channels, kernel_size=3, padding='valid', strides=strides)
        self.conv3 = tf.keras.layers.Conv2D(num_channels, kernel_size=1)

        if use_1x1conv:
            self.conv4 = tf.keras.layers.Conv2D(num_channels, kernel_size=1, strides=strides)
        else:
            self.conv4 = None

        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.bn3 = tf.keras.layers.BatchNormalization()

    def call(self, inputs, training=None):
        X = inputs
        Y = tf.nn.relu(self.bn1(self.conv1(X), training=training))
        paddings = tf.constant([[0, 0], [1, 1], [1, 1], [0, 0]])
        Y = tf.pad(Y, paddings)
        Y = self.bn2(self.conv2(Y), training=training)
        Y = self.bn3(self.conv3(Y), training=training)

        if self.conv4:
            X = self.conv4(X)

        return tf.nn.relu(Y + X)

# InitialBlock

This constructs the very first layers of the network which are crucial for reducing spatial dimensions and increasing depth (number of channels) before entering the main residual blocks.

In [ ]:
# Initial block with convolution, batch norm, ReLU, and max pooling
class InitialBlock(tf.keras.layers.Layer):
    def __init__(self):
        super(InitialBlock, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='valid')
        #valid padding with additional pads before to be equiv to nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()
        self.pool = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='valid')
        #valid padding with additional pads before to be equiv to nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def call(self, inputs, training=None):
        paddings = tf.constant([[0, 0], [3, 3], [3, 3], [0, 0]])
        inputs = tf.pad(inputs, paddings)
        x = self.conv(inputs)
        x = self.bn(x, training=training)
        x = self.relu(x)
        pool_paddings = tf.constant([[0, 0], [1, 1], [1, 1], [0, 0]])
        x = tf.pad(x, pool_paddings)
        x = self.pool(x)
        return x

# ResNet Block Function
This function uses the Residual class to construct sequences of residual blocks, which are the core components of ResNet architectures. It organizes these blocks based on their position in the network, adjusting their properties for downsampling or maintaining dimensionality where necessary.

In [ ]:
def resnet_block(input_channels, min_channels, num_channels, num_residuals, stride,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, min_channels, num_channels,
                                use_1x1conv=True, strides=stride))
        elif i == 0 and first_block:
            blk.append(Residual(input_channels, min_channels, num_channels, use_1x1conv=True))
        else:
            blk.append(Residual(num_channels, min_channels, num_channels))
    return blk

# MyResNet50

This class integrates all previously defined components into a complete network. It uses the InitialBlock to process the input initially, then sequentially passes the data through multiple resnet_block sequences to form the deep architecture typical of ResNets. It concludes with global average pooling and a fully connected layer to produce the final output.

In [ ]:
class MyResNet50(tf.keras.Model):
    def __init__(self):
        super(MyResNet50, self).__init__()
        self.b1 = InitialBlock()
        self.b2 = tf.keras.Sequential(resnet_block(64, 64, 256, 3, 2, first_block=True))
        self.b3 = tf.keras.Sequential(resnet_block(256, 128, 512, 4, 2))
        self.b4 = tf.keras.Sequential(resnet_block(512, 256, 1024, 6, 2))
        self.b5 = tf.keras.Sequential(resnet_block(1024, 512, 2048, 2, 2))

        self.avg_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.flat = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes) #needs to be changed based on the train set's num classes

    def call(self, inputs, training=None):
        x = self.b1(inputs, training=training)
        x = self.b2(x, training=training)
        x = self.b3(x, training=training)
        x = self.b4(x, training=training)
        x = self.b5(x, training=training)
        x = self.avg_pool(x)
        x = self.flat(x)
        x = self.fc(x)
        return x

# Weight Initialization

In [ ]:
# Weight Initialization
model = MyResNet50()
model.build((batch_size, 128, 128, 3))  # Assuming input images are 128x128 with 3 channels
model.summary()
initializer = tf.keras.initializers.GlorotUniform()
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
        layer.kernel_initializer = initializer

Model: "my_res_net50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 initial_block (InitialBloc  multiple                  9728      
 k)                                                              
                                                                 
 sequential (Sequential)     (256, 32, 32, 256)        219008    
                                                                 
 sequential_1 (Sequential)   (256, 16, 16, 512)        1228288   
                                                                 
 sequential_2 (Sequential)   (256, 8, 8, 1024)         7124992   
                                                                 
 sequential_3 (Sequential)   (256, 4, 4, 2048)         10518528  
                                                                 
 global_average_pooling2d (  multiple                  0         
 GlobalAveragePooling2D)                              

# Adjusting Learning Rate Based on Schedule

Removed because we are using Adam, which is already able to dynamically change the learning rate.

In [ ]:
# class CustomCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
#     def __init__(self, initial_lr, total_epochs):
#         super().__init__()
#         self.initial_lr = tf.constant(initial_lr, dtype=tf.float32)
#         self.total_epochs = tf.constant(total_epochs, dtype=tf.float32)  # Ensure total_epochs is also a float
#         self.pi = tf.constant(math.pi, dtype=tf.float32)

#     def __call__(self, step):
#         step = tf.cast(step, tf.float32)  # Ensure step is treated as float
#         return self.initial_lr * 0.5 * (1 + tf.cos(self.pi * step / self.total_epochs))

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Accuracy

In [ ]:
def accuracy(y_pred, y_true):
    predicted_indices = tf.argmax(y_pred, axis=1)
    true_indices = tf.argmax(y_true, axis=1)  # Assuming y_true is one-hot encoded
    correct_predictions = tf.equal(predicted_indices, true_indices)
    return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

# Train

In [ ]:
@tf.function
def apply_grads(opt, grads, tv):
    opt.apply_gradients(zip(grads, tv))

def train_step(model, images, labels, loss_fn, optimizer):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_fn(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    apply_grads(optimizer, gradients, model.trainable_variables)
    acc = accuracy(predictions, labels)
    return tf.cast(loss, tf.float32), tf.cast(acc, tf.float32)

def train(model, data_loader, optimizer, epoch, initial_lr, total_epochs, loss_fn):
    #optimizer.learning_rate = CustomCosineDecay(initial_lr, total_epochs)  # Adjust learning rate
    #optimizer.learning_rate = cos_decay(epoch)  # Adjust learning rate
    total_loss, total_num, trainacc = 0.0, 0.0, 0.0
    train_bar = tqdm(data_loader, ncols=100)
    for images, labels in train_bar:
        if images.shape[0] != labels.shape[0]:
          print(f"Shape mismatch: {images.shape} vs {labels.shape}")
          continue  # Skip this batch or handle error
        if images.shape[0] == 0:
            print("Warning: Received an empty batch of images.")
            continue
        if labels.shape[0] == 0:
            print("Warning: Received an empty batch of labels.")
            continue
        loss, acc = train_step(model, images, labels, loss_fn, optimizer)
        batch_size = tf.cast(tf.shape(images)[0], tf.float32)  # Ensure it's float
        if batch_size == 0:
            print("Warning: Batch size is zero.")
            continue
        trainacc += acc * batch_size
        total_num += batch_size
        total_loss += loss * batch_size
        train_bar.set_description(
            f'Train Epoch: [{epoch}/{total_epochs}], Loss: {total_loss / total_num:.4f}, trainacc: {trainacc / total_num:.6f}')

    return total_loss / total_num, trainacc / total_num
#optimizer = tf.keras.optimizers.SGD(learning_rate=CustomCosineDecay(args.lr, args.epochs), momentum=0.9)
#optimizer = tf.keras.optimizers.Adam(learning_rate=CustomCosineDecay(args.lr, args.epochs))
optimizer = tf.keras.optimizers.Adam(args.lr)

# Test

In [ ]:
def test(model, test_data_loader, epoch, total_epochs):
    testacc, total_num, test_bar = 0.0, 0, tqdm(test_data_loader)
    for image, label in test_bar:
        predictions = model(image, training=False)
        acc = accuracy(predictions, label)
        total_num += image.shape[0]
        testacc += acc * image.shape[0]
        test_bar.set_description(
            f'Test Epoch: [{epoch}/{total_epochs}], testacc: {testacc / total_num:.6f}')
    return testacc / total_num

# Testing On Picture

In [ ]:
# Results dictionary to store metrics
# results = {'train_loss': [], 'train_acc': [], 'test_acc': [], 'lr': []}
results = {'train_loss': [], 'train_acc': [], 'test_acc': []}
epoch_start = 1

# Load model and optimizer states if resuming from a checkpoint
if args.resume:
    checkpoint = tf.train.Checkpoint(model, optimizer=optimizer)
    status = checkpoint.restore(tf.train.latest_checkpoint(args.resume))
    status.assert_existing_objects_matched()
    #epoch_start = ?
    print(f'Loaded from: {args.resume}')
else:
    # If not resuming, initialize weights
    model.build((batch_size, 128, 128, 3))  # Assuming input images are 128x128 with 3 channels
    initializer = tf.keras.initializers.GlorotUniform()
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense)):
            layer.kernel_initializer = initializer
            if hasattr(layer, 'kernel'):
                layer.kernel.assign(initializer(shape=layer.kernel.shape))

# Ensure the results directory exists
if not os.path.exists(directory):
    os.makedirs(directory)

# Save arguments to a JSON file
with open(os.path.join(directory, 'args.json'), 'w') as fid:
    json.dump(vars(args), fid, indent=2)

# Training loop
for epoch in range(epoch_start, args.epochs + 1):
    train_loss, train_acc = train(model, train_dataset1, optimizer, epoch, args.lr, args.epochs,loss_fn)
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_acc)
    test_acc = test(model, test_dataset1, epoch,args.epochs)
    results['test_acc'].append(test_acc)
    #current_lr = args.lr * 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    #results['lr'].append(current_lr)
    #optimizer.lr.assign(current_lr)  # Adjust learning rate

    # Save statistics to CSV
    data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    data_frame.to_csv(os.path.join(directory, 'log.csv'), index_label='epoch')

    # Save checkpoints at specified frequency
    if epoch % args.checkpoint_freq == 0:
        checkpoint_path = os.path.join(directory, f'checkpoint_ep{epoch:04}.ckpt')
        checkpoint = tf.train.Checkpoint(model, optimizer=optimizer, epoch=tf.Variable(epoch))
        checkpoint.save(file_prefix=checkpoint_path)

print('Training complete')

Train Epoch: [1/50], Loss: 7.1725, trainacc: 0.006184: 100%|██████| 238/238 [03:16<00:00,  1.21it/s]
Test Epoch: [1/50], testacc: 0.001600: 100%|██████████| 64/64 [01:21<00:00,  1.28s/it]
Train Epoch: [2/50], Loss: 6.4963, trainacc: 0.012927: 100%|██████| 238/238 [01:52<00:00,  2.12it/s]
Test Epoch: [2/50], testacc: 0.001846: 100%|██████████| 64/64 [00:19<00:00,  3.30it/s]
Train Epoch: [3/50], Loss: 5.6665, trainacc: 0.033928: 100%|██████| 238/238 [02:21<00:00,  1.68it/s]
Test Epoch: [3/50], testacc: 0.006583: 100%|██████████| 64/64 [00:20<00:00,  3.14it/s]
Train Epoch: [4/50], Loss: 4.7619, trainacc: 0.091358: 100%|██████| 238/238 [01:59<00:00,  1.99it/s]
Test Epoch: [4/50], testacc: 0.006214: 100%|██████████| 64/64 [00:19<00:00,  3.26it/s]
Train Epoch: [5/50], Loss: 3.9056, trainacc: 0.191695: 100%|██████| 238/238 [01:52<00:00,  2.11it/s]
Test Epoch: [5/50], testacc: 0.029285: 100%|██████████| 64/64 [00:20<00:00,  3.19it/s]
Train Epoch: [6/50], Loss: 3.1922, trainacc: 0.306883: 100%|

KeyboardInterrupt: 